# Tensor

In [2]:
import pandas as pd

import tensorflow as tf

In [3]:
lookup = pd.read_pickle('./data/lookup')
batches = pd.read_pickle('./data/batches')

In [4]:
tf.reset_default_graph()
g = tf.get_default_graph()

X = tf.placeholder(dtype=tf.int32, shape=[None, 150])
Y = tf.placeholder(dtype=tf.float32, shape=[None, 1])

attentions, _ = tf.nn.bidirectional_dynamic_rnn(
    tf.nn.rnn_cell.BasicLSTMCell(32),
    tf.nn.rnn_cell.BasicLSTMCell(32),
    tf.nn.embedding_lookup(tf.Variable(lookup, dtype=tf.float32), X),
    dtype=tf.float32
)
attention = tf.concat(attentions, axis=2)
F = tf.reshape(
    tf.layers.conv1d(
        tf.transpose(attention, [0, 2, 1]),
        64,
        64
    ),
    shape=[-1, 64]
)

L = tf.layers.dense(F, 16, activation=tf.sigmoid)
output = tf.layers.dense(L, 1, activation=tf.sigmoid)

loss = tf.losses.mean_squared_error(Y, output)
learn = tf.train.AdamOptimizer().minimize(loss)

init = tf.global_variables_initializer()

sess = tf.Session(graph=g)
sess.run(init)

In [13]:
for iter_ in range(len(batches)):
    
    feed = {
        X: batches[iter_]['X'],
        Y: batches[iter_]['Y']
    }
    
    loss_, _ = sess.run([loss, learn], feed_dict=feed)
    iter_ += 1
    if not iter_ % 20:
        print(loss_)

0.02044429
0.020140618
0.019734213
0.01988582
0.020440796
0.019544084
0.020156655
0.019822568
0.018898854
0.019460408
0.01862282
0.019632142
0.019941906
0.018739276
0.01911088
0.018585254


In [12]:
saver = tf.train.Saver()
saver.save(sess, './data/checkpoint/model')

print(sess.run(output, feed_dict=feed)[-1])

[0.20854172]
